In [1]:
import pandas as pd
import numpy as np
import os
import seaborn as sns
import matplotlib.pyplot as plt
from collections import defaultdict
import plotly.express as px
import plotly.offline as go

In [2]:
df = pd.read_csv('funnel_df.csv')

In [3]:
df.head()

,user_id,date,device,sex,home_page,search_page,payment_page,payment_confirmation_page
0,450007,2015-02-28,Desktop,Female,1,0,0,0
1,756838,2015-01-13,Desktop,Male,1,0,0,0
2,568983,2015-04-09,Desktop,Male,1,1,0,0
3,190794,2015-02-18,Desktop,Female,1,1,0,0
4,537909,2015-01-15,Desktop,Male,1,0,0,0


In [4]:
cols = ["home_page","search_page","payment_page","payment_confirmation_page"]
df[cols] = df[cols].replace({'0':np.nan, 0:np.nan})

In [5]:
df.head()

,user_id,date,device,sex,home_page,search_page,payment_page,payment_confirmation_page
0,450007,2015-02-28,Desktop,Female,1,NaN,NaN,NaN
1,756838,2015-01-13,Desktop,Male,1,NaN,NaN,NaN
2,568983,2015-04-09,Desktop,Male,1,1.0,NaN,NaN
3,190794,2015-02-18,Desktop,Female,1,1.0,NaN,NaN
4,537909,2015-01-15,Desktop,Male,1,NaN,NaN,NaN


In [6]:
df.apply(pd.value_counts)


,user_id,date,device,sex,home_page,search_page,payment_page,payment_confirmation_page
1,NaN,NaN,NaN,NaN,90400.0,45200.0,6030.0,452.0
17,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
28,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
37,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
38,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
2015-04-30,NaN,801.0,NaN,NaN,NaN,NaN,NaN,NaN
Desktop,NaN,NaN,60200.0,NaN,NaN,NaN,NaN,NaN
Female,NaN,NaN,NaN,45075.0,NaN,NaN,NaN,NaN
Male,NaN,NaN,NaN,45325.0,NaN,NaN,NaN,NaN


In [7]:
data = dict(
    number = [90400,45200,6030,452],
    stage = ['home_page','search_page','payment_page','payment_confirmation_page'])

In [8]:
fig = px.funnel(data, x='number', y='stage',title='Funnel')
fig.show()

In [9]:
plot_url = (fig, 'fig1', auto_open=False)

print('url: ' + plot_url)

SyntaxError: invalid syntax (3333531059.py, line 1)

In [ ]:
df_female = df[df.sex != 'Male']
df_female.apply(pd.value_counts)

In [ ]:
data = dict(
    number = [45075,22676,3100,241],
    stage = ['home_page','search_page','payment_page','payment_confirmation_page'])
fig = px.funnel(data, x='number', y='stage', title = 'Female Funnel')
fig.show()

In [ ]:
df_male = df[df.sex != 'Female']
df_male.apply(pd.value_counts)

In [ ]:
data = dict(
    number = [45325,22524,2930,211],
    stage = ['home_page','search_page','payment_page','payment_confirmation_page'])
fig = px.funnel(data, x='number', y='stage', title = 'Male Funnel')
fig.show()

In [ ]:
df_desktop = df[df.device != 'Mobile']
df_mobile = df[df.device != 'Desktop']

In [ ]:
df_desktop.apply(pd.value_counts)

In [ ]:
df_mobile.apply(pd.value_counts)

In [ ]:
data = dict(
    number = [60200,30100,3010,150],
    stage = ['home_page','search_page','payment_page','payment_confirmation_page'])
fig = px.funnel(data, x='number', y='stage', title = 'Desktop Funnel')
fig.show()

In [ ]:
data = dict(
    number = [30200,15100,3020.0,302.0],
    stage = ['home_page','search_page','payment_page','payment_confirmation_page'])
fig = px.funnel(data, x='number', y='stage', title = 'Mobile Funnel')
fig.show()

In [ ]:
df['date'] =  pd.to_datetime(df['date'])

In [ ]:
df2 = pd.DataFrame({'date': pd.date_range('2015-01-01', '2015-04-30')})

In [ ]:
def trends_over_time(df, df_column, new_df_name, count_name):
    # df is a pandas dataframe
    # df_column is a string naming a column contained in df
    # new_df_name is a string that names the resulting dataframe
    # count name is a string that names the label parameter on the graph and column in new_df_name dataframe
    new_df_name = df[df[df_column] == 1]
    new_df_name = new_df_name[['date']]
    new_df_name['date_count'] = new_df_name.date.map(new_df_name.groupby('date').size())
    new_df_name = new_df_name.sort_values(by='date').reset_index()
    new_df_name = new_df_name[['date','date_count']]
    new_df_name=new_df_name.drop_duplicates()
    idx = pd.date_range('2015-01-01', '2015-04-30')
    new_df_name = new_df_name.set_index('date')
    s = new_df_name.squeeze()
    s.index = pd.DatetimeIndex(s.index)
    s = s.reindex(idx, fill_value=0)
    new_df_name = s.to_frame()
    new_df_name = new_df_name.reset_index(level=0)
    new_df_name.rename(columns={'index':'date','date_count': count_name},inplace=True)
    df2[count_name] = new_df_name[count_name]
    new_df_name.plot(x='date', y=count_name,kind="line")
    plt.show()

In [ ]:
trends_over_time(df,"home_page","df_homepage",'homepage_count')

In [ ]:
trends_over_time(df,"search_page","df_search",'searchpage_count')

In [ ]:
trends_over_time(df,"payment_page","df_payment_page",'payment_page_count')

In [ ]:
trends_over_time(df,"payment_confirmation_page","df_confirm",'payment_confirmation_count')

In [ ]:
df2.describe()

In [ ]:
df2.to_csv('date_counts.csv')